In [1]:
import argparse
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import random
import numpy as np
import torch.nn as nn
import torch
from transformers import AutoModel
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score, classification_report
from transformers import AdamW, get_linear_schedule_with_warmup
# Any results you write to the current directory are saved as output.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from transformers import BertTokenizer,BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pack_padded_sequence
from torch.optim import AdamW
from tqdm import tqdm
from argparse import ArgumentParser
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.engine.engine import Engine, State, Events
from ignite.handlers import EarlyStopping
from ignite.contrib.handlers import TensorboardLogger, ProgressBar
from ignite.utils import convert_tensor
from torch.optim.lr_scheduler import ExponentialLR
import warnings  
warnings.filterwarnings('ignore')
import gc
import copy
import time
import random
import string

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold
from transformers import AutoTokenizer, AutoModel, AdamW
import random
import os
from urllib import request
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Helper Function

In [2]:
# Define a dataset class for prompted BERT fine tuning
# The dataset takes in query and passage, and then construct a training sample as:  <prompt> + [MASK] + <text>, returning the position of the mask
# It also performs padding and stores the labels
class BERTPromptDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, labels, tokenizer, prompt="ما هو شعور الكاتب؟", max_length=512):
        # Construct the input sentence
        input_sentences = ["{} {} {}".format(prompt, tokenizer.mask_token, text) for _, (text) in dataset.iterrows()]
        
        # Encode and store
        encodings_dict = tokenizer.batch_encode_plus(input_sentences, truncation=True, max_length=max_length, padding="max_length")
        self.input_ids = encodings_dict['input_ids']
        self.attn_masks = encodings_dict['attention_mask']
        self.labels = labels

        # Calculate the position of the mask using self.input_ids
        mask_id = tokenizer.encode(tokenizer.mask_token)[1] # 103
        self.mask_pos = [sent_ids.index(mask_id) for sent_ids in self.input_ids]
        
        
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return_dict = {"text_ids": torch.tensor(self.input_ids[idx]),
                       "text_mask": torch.tensor(self.attn_masks[idx]), 
                       "mask_pos": torch.tensor(self.mask_pos[idx]),
                       "labels": torch.tensor(self.labels[idx]).float()} 
        return return_dict

In [3]:
# Class that expects a prompted input from the BERTPromptDataset
# Takes the input, forward propagates it through BERT and concatenates the output at the [MASK] and [CLS] token to get a representation of the text
# This is then passed to a linear head to perform binary classification for how relevant it is
class BERTPrompt(torch.nn.Module):
    def __init__(self, bert, tokenizer):
        super().__init__()
        self.bert = bert.cuda()
        self.tokenizer = tokenizer
        self.linear = torch.nn.Linear(768*2, 3).cuda()
        self.act = torch.nn.Softmax()
        
    # input_dict is obtained through indexing the dataset e.g. dataset[0:10]
    def forward(self, input_ids,attention_mask,mask_pos ):
        output = self.bert(input_ids,attention_mask)[0] # output is of shape [10, 256, 768]
        cls_out = output[:, 0, :]
        mask_out = output[torch.arange(cls_out.shape[0]), mask_pos, :] # indexing like [[0, 1], [13, 14]] will select items [[0, 13], [1, 14]]
        
        representation = torch.cat([cls_out, mask_out], dim=1)
        logit = self.linear(representation)
        return logit
    
#     # return the logit with sigmoid activation applied
#     def predict(self, input_dict):
#         return self.forward(input_dict)

In [4]:
from typing import Optional, Sequence

import torch
from torch import Tensor
from torch import nn
from torch.nn import functional as F


class FocalLoss(nn.Module):
    """ Focal Loss, as described in https://arxiv.org/abs/1708.02002.
    It is essentially an enhancement to cross entropy loss and is
    useful for classification tasks when there is a large class imbalance.
    x is expected to contain raw, unnormalized scores for each class.
    y is expected to contain class labels.
    Shape:
        - x: (batch_size, C) or (batch_size, C, d1, d2, ..., dK), K > 0.
        - y: (batch_size,) or (batch_size, d1, d2, ..., dK), K > 0.
    """

    def __init__(self,
                 alpha: Optional[Tensor] = None,
                 gamma: float = 0.,
                 reduction: str = 'mean',
                 ignore_index: int = -100):
        """Constructor.
        Args:
            alpha (Tensor, optional): Weights for each class. Defaults to None.
            gamma (float, optional): A constant, as described in the paper.
                Defaults to 0.
            reduction (str, optional): 'mean', 'sum' or 'none'.
                Defaults to 'mean'.
            ignore_index (int, optional): class label to ignore.
                Defaults to -100.
        """
        if reduction not in ('mean', 'sum', 'none'):
            raise ValueError(
                'Reduction must be one of: "mean", "sum", "none".')

        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction

        self.nll_loss = nn.NLLLoss(
            weight=alpha, reduction='none', ignore_index=ignore_index)

    def __repr__(self):
        arg_keys = ['alpha', 'gamma', 'ignore_index', 'reduction']
        arg_vals = [self.__dict__[k] for k in arg_keys]
        arg_strs = [f'{k}={v}' for k, v in zip(arg_keys, arg_vals)]
        arg_str = ', '.join(arg_strs)
        return f'{type(self).__name__}({arg_str})'

    def forward(self, x: Tensor, y: Tensor) -> Tensor:
        if x.ndim > 2:
            # (N, C, d1, d2, ..., dK) --> (N * d1 * ... * dK, C)
            c = x.shape[1]
            x = x.permute(0, *range(2, x.ndim), 1).reshape(-1, c)
            # (N, d1, d2, ..., dK) --> (N * d1 * ... * dK,)
            y = y.view(-1)

        unignored_mask = y != self.ignore_index
        y = y[unignored_mask]
        if len(y) == 0:
            return torch.tensor(0.)
        x = x[unignored_mask]

        # compute weighted cross entropy term: -alpha * log(pt)
        # (alpha is already part of self.nll_loss)
        log_p = F.log_softmax(x, dim=-1)
        ce = self.nll_loss(log_p, y)

        # get true class column from each row
        all_rows = torch.arange(len(x))
        log_pt = log_p[all_rows, y]

        # compute focal term: (1 - pt)^gamma
        pt = log_pt.exp()
        focal_term = (1 - pt)**self.gamma

        # the full loss: -alpha * ((1 - pt)^gamma) * log(pt)
        loss = focal_term * ce

        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()

        return loss


def focal_loss(alpha: Optional[Sequence] = None,
               gamma: float = 0.,
               reduction: str = 'mean',
               ignore_index: int = -100,
               device='cuda',
               dtype=torch.float32) -> FocalLoss:
    """Factory function for FocalLoss.
    Args:
        alpha (Sequence, optional): Weights for each class. Will be converted
            to a Tensor if not None. Defaults to None.
        gamma (float, optional): A constant, as described in the paper.
            Defaults to 0.
        reduction (str, optional): 'mean', 'sum' or 'none'.
            Defaults to 'mean'.
        ignore_index (int, optional): class label to ignore.
            Defaults to -100.
        device (str, optional): Device to move alpha to. Defaults to 'cpu'.
        dtype (torch.dtype, optional): dtype to cast alpha to.
            Defaults to torch.float32.
    Returns:
        A FocalLoss object
    """
    if alpha is not None:
        if not isinstance(alpha, Tensor):
            alpha = torch.tensor(alpha)
        alpha = alpha.to(device=device, dtype=dtype)

    fl = FocalLoss(
        alpha=alpha,
        gamma=gamma,
        reduction=reduction,
        ignore_index=ignore_index)
    return fl

In [5]:
class F1_Loss(nn.Module):
    '''Calculate F1 score. Can work with gpu tensors
    
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Returns
    -------
    torch.Tensor
        `ndim` == 1. epsilon <= val <= 1
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    - http://www.ryanzhang.info/python/writing-your-own-loss-function-module-for-pytorch/
    '''
    def __init__(self, epsilon=1e-7):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true,):
        # assert y_pred.ndim == 2
        # assert y_true.ndim == 1
        # print(y_pred.shape)
        # print(y_true.shape)
        # y_pred[y_pred<0.5]=0
        # y_pred[y_pred>=0.5]=0


        
        y_true_one_hot = F.one_hot(y_true.to(torch.int64), 3).to(torch.float32)
        # y_pred_one_hot = F.one_hot(y_pred.to(torch.int64), 2).to(torch.float32)
        
        tp = (y_true_one_hot * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true_one_hot) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true_one_hot) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true_one_hot * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        f1=f1.detach()
        # print(f1.shape)
        # y_pred=y_pred.reshape((y_pred.shape[0], 1))
        # y_true=y_true.reshape((y_true.shape[0], 1))

        # p1=y_true*(math.log(sigmoid(y_pred)))*(1-f1)[1]
        # p0=(1-y_true)*math.log(1-sigmoid(y_pred))*(1-f1)[0]


        # y_true_one_hot = F.one_hot(y_true.to(torch.int64), 2)
        # print(y_pred)
        # print(y_true_one_hot)
        CE =torch.nn.CrossEntropyLoss(weight=( 1 - f1))(y_pred, y_true)
        # loss = ( 1 - f1)  * CE
        return  CE.mean()

In [6]:
def criterion(outputs1,  targets):

    criterion = F1_Loss()
    criterion_2 = FocalLoss()
    loss = 0.5*criterion(outputs1, targets)+0.5*criterion_2(outputs1, targets)
    return loss

In [7]:
def run_training(model, optimizer, scheduler, device, num_epochs, train_loader, valid_loader):
    # To automatically log gradients
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = 0
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss,f1_score = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        
       
        
        # deep copy the model
        if f1_score >= best_epoch_loss:
            print(f"Validation Loss Improved ({best_epoch_loss} ---> {f1_score})")
            best_epoch_loss = f1_score
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = "/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/araelectra_prompt/Loss_new.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print("Model Saved")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [8]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [9]:
def prepare_loaders(train,valid):
   
    
    train_dataset = BERTPromptDataset(train[['text']], train['label'], tokenizer)
    valid_dataset = BERTPromptDataset(valid[['text']], valid['label'], tokenizer)
    
    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

    
    
    

In [10]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    prediction=[]
    true_prediction=[]
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        
        text_ids = data['text_ids'].to(device, dtype = torch.long)
        text_mask = data['text_mask'].to(device, dtype = torch.long)
        mask_pos =data['mask_pos'].to(device, dtype = torch.long)
        targets = data['labels'].to(device, dtype=torch.long)
        
        batch_size = text_ids.size(0)

        outputs = model(text_ids, text_mask,mask_pos)
        prediction.append(F.softmax(outputs).to('cpu').numpy())
        true_prediction.append(targets.to('cpu').numpy())

        # outputs = outputs.argmax(dim=1)
        
        loss = criterion(outputs, targets)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr']) 
    
    gc.collect()
#     print(np.array(prediction)[0][0])
    prediction=np.argmax(np.array(prediction)[0],axis=1)
    print(print_statistics(np.array(true_prediction)[0],prediction))
    print(f1_score(np.array(true_prediction)[0],prediction, average='macro'))

    
    return epoch_loss, f1_score(np.array(true_prediction)[0],prediction, average='macro')

In [11]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        
        text_ids = data['text_ids'].to(device, dtype = torch.long)
        text_mask = data['text_mask'].to(device, dtype = torch.long)
        mask_pos =data['mask_pos'].to(device, dtype = torch.long)
        targets = data['labels'].to(device, dtype=torch.long)
        
        batch_size = text_ids.size(0)
        # print(targets)

        outputs = model(text_ids, text_mask,mask_pos)
        # print(outputs.shape)
        
        # print(outputs.shape)

        
        loss = criterion(outputs, targets)
        loss = loss / CONFIG['n_accumulate']
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss

In [12]:
from sklearn.metrics import jaccard_score,f1_score,accuracy_score,recall_score,precision_score,classification_report
def print_statistics(y, y_pred):
    accuracy = accuracy_score(y, y_pred)
    precision =precision_score(y, y_pred, average='weighted')
    recall = recall_score(y, y_pred, average='weighted')
    f_score = f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(classification_report(y, y_pred))
    return accuracy, precision, recall, f_score

# Begin Training

In [13]:
train = pd.read_csv('/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/Dataset/NADI2022_Subtask2_TRAIN.tsv', sep='\t', lineterminator='\n')
valid = pd.read_csv('/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/Dataset/NADI2022_Subtask2_DEV.tsv', sep='\t', lineterminator='\n')

In [14]:
CONFIG = {"seed": 42,
          "epochs": 15,
          "train_batch_size": 8,
          "valid_batch_size": 64,
          "max_length": 512,
          "learning_rate": 2e-5,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-8,
          "T_max": 500,
          "weight_decay": 1e-8,
          "n_fold": 5,
          "n_accumulate": 1,
          "num_classes": 18,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          }

In [15]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['#3_label'] = le.fit_transform(train['#3_label'].values)
valid['#3_label'] = le.transform(valid['#3_label'].values)
train.rename(columns = {'#2_content':'text', '#3_label':'label'}, inplace = True)
valid.rename(columns = {'#2_content':'text', '#3_label':'label'}, inplace = True)


In [16]:
tokenizer= AutoTokenizer.from_pretrained('aubmindlab/araelectra-base-discriminator')


In [17]:
arabert_model=AutoModel.from_pretrained('aubmindlab/araelectra-base-discriminator')


Some weights of the model checkpoint at aubmindlab/araelectra-base-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
arabert_prompt = BERTPrompt(arabert_model, tokenizer)


In [20]:


    
# Create Dataloaders
train_loader, valid_loader = prepare_loaders(train,valid)

arabert_prompt.to(CONFIG['device'])
torch.cuda.empty_cache()

# Define Optimizer and Scheduler
no_decay = ["bias", "LayerNorm.weight"]
param_optimizer = list(arabert_prompt.named_parameters())
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay) and p[1].requires_grad],
         "lr":1e-5,
        "weight_decay": 1e-2,
    },
    {
        "params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay) and p[1].requires_grad],
         "lr":1e-3,
        "weight_decay": 0.0,
    },
]

    
optimizer = AdamW(optimizer_grouped_parameters, lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
scheduler = fetch_scheduler(optimizer)

model, history = run_training(arabert_prompt, optimizer, scheduler,
                              device=CONFIG['device'],
                              num_epochs=CONFIG['epochs'],
                              train_loader=train_loader, valid_loader=valid_loader )


del model, history, train_loader, valid_loader
_ = gc.collect()
print()

[INFO] Using GPU: NVIDIA GeForce RTX 3080 Ti

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|███████████████████████████████████████████| 187/187 [00:30<00:00,  6.28it/s, Epoch=1, LR=6.93e-6, Train_Loss=1.05]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|███████████████████████████████████████████| 187/187 [00:30<00:00,  6.18it/s, Epoch=1, LR=6.93e-6, Train_Loss=1.05]
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████████████████████████████████████| 187/187 [00:30<00:00,  6.09it/s, Epoch=15, LR=6.7e-6, Train_Loss=0.337]
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|███████████████████████████████████████████████| 8/8 [00:02<00:00,  2.76it/s, Epoch=15, LR=6.7e-6, Valid_Loss=1.73]

Accuracy: 0.656
Precision: 0.708
Recall: 0.656
F_score: 0.643

              precision    recall  f1-score   support

           0       0.78      0.58      0.67        24
           1       0.83      0.36      0.50        14
           2       0.57      0.88      0.70        26

    accuracy                           0.66        64
   macro avg       0.73      0.61      0.62        64
weighted avg       0.71      0.66      0.64        64

(0.65625, 0.7075520833333333, 0.65625, 0.6425189393939394)
0.6212121212121212

Training complete in 0h 8m 28s
Best Loss: 0.6290



In [19]:


    
# Create Dataloaders
train_loader, valid_loader = prepare_loaders(train,valid)

arabert_prompt.to(CONFIG['device'])
torch.cuda.empty_cache()

# Define Optimizer and Scheduler
optimizer = AdamW(arabert_prompt.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
scheduler = fetch_scheduler(optimizer)

model, history = run_training(arabert_prompt, optimizer, scheduler,
                              device=CONFIG['device'],
                              num_epochs=CONFIG['epochs'],
                              train_loader=train_loader, valid_loader=valid_loader )


del model, history, train_loader, valid_loader
_ = gc.collect()
print()

[INFO] Using GPU: NVIDIA GeForce RTX 3080 Ti

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|███████████████████████████████████████████| 187/187 [00:30<00:00,  6.23it/s, Epoch=1, LR=1.39e-5, Train_Loss=1.06]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|███████████████████████████████████████████| 187/187 [00:30<00:00,  6.15it/s, Epoch=1, LR=1.39e-5, Train_Loss=1.06]
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|██████████████████████████████████████████████| 8/8 [00:02<00:00,  2.94it/s, Epoch=15, LR=1.34e-5, Valid_Loss=2.53]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████████████████████████████████████████| 8/8 [00:02<00:00,  2.79it/s, Epoch=15, LR=1.34e-5, Valid_Loss=2.53]

Accuracy: 0.547
Precision: 0.614
Recall: 0.547
F_score: 0.556

              precision    recall  f1-score   support

           0       0.71      0.42      0.53        24
           1       0.35      0.64      0.45        14
           2       0.67      0.62      0.64        26

    accuracy                           0.55        64
   macro avg       0.58      0.56      0.54        64
weighted avg       0.61      0.55      0.56        64

(0.546875, 0.61441163003663, 0.546875, 0.5558059210526316)
0.5387719298245615

Training complete in 0h 8m 28s
Best Loss: 0.6288

